In [1]:
from file_encoder import FileEncoder
from data_manager import DataManager
import numpy as np
import sys
import tensorflow as tf
from tensorflow.keras import layers, models, Model

TARGET_DIR = "../thirdparty/「波音リツ」歌声データベースVer2/DATABASE"
OUTPUT_DIR = "../master/ust/json"


if __name__ == "__main__":
    need_encode = True

    if len(sys.argv) > 1:
        if sys.argv[1] == "--encode":
            need_encode = True
    
    manager = DataManager()

    if need_encode:
        encoder = FileEncoder(TARGET_DIR, OUTPUT_DIR)
        names, lyric_indexs, durations, notenums, y = encoder.encode()
        manager.save(lyric_indexs, durations, notenums, names, y)
    
    lyric_indexs, duration_indexs, notenum_indexs, names, y = manager.load()

    assert len(y) > 0 and len(lyric_indexs) > 0 and len(duration_indexs) > 0 and len(notenum_indexs) > 0

    """
    print(f"lyric_indexs.shape: {lyric_indexs.shape}")
    print(f"lyric_indexs: {lyric_indexs}")
    print(f"duration_indexs.shape: {duration_indexs.shape}")
    print(f"duration_indexs: {duration_indexs}")
    print(f"notenum_indexs.shape: {notenum_indexs.shape}")
    print(f"notenum_indexs: {notenum_indexs}")
    print(f"y.shape: {y.shape}")
    print(f"y: {y}")

    print(f"train_lyric.shape: {train_lyric.shape}")
    print(f"train_duration.shape: {train_duration.shape}")
    print(f"train_notenum.shape: {train_notenum.shape}")
    print(f"y_train.shape: {y_train.shape}")
    print(f"test_lyric.shape: {test_lyric.shape}")
    print(f"test_duration.shape: {test_duration.shape}")
    print(f"test_notenum.shape: {test_notenum.shape}")
    print(f"y_test.shape: {y_test.shape}")
    
    """


    (train_lyric, train_duration, train_notenum), (test_lyric, test_duration, test_notenum), y_train, y_test = manager.get_train_and_test_data()
    assert train_lyric.shape[0] == train_duration.shape[0] == train_notenum.shape[0] == y_train.shape[0]
    assert test_lyric.shape[0] == test_duration.shape[0] == test_notenum.shape[0] == y_test.shape[0]

    emmbed_lyric_dim = np.max(lyric_indexs) + 1
    print(f"emmbed_lyric_dim: {emmbed_lyric_dim}")
    emmbed_duration_dim = np.max(duration_indexs) + 1
    print(f"emmbed_duration_dim: {emmbed_duration_dim}")
    emmbed_notenum_dim = np.max(notenum_indexs) + 1
    print(f"emmbed_notenum_dim: {emmbed_notenum_dim}")

    max_lyric_index = np.max(lyric_indexs)

    train_x = [train_lyric, train_duration, train_notenum]

    print("timestep", y_train.shape[1])
    print("y_train.shape[1:]", y_train.shape[1:])

    print("train_lyric.shape[1:]", train_lyric.shape[1:])
    
    lyric_input = tf.keras.Input(shape=train_lyric.shape[1:], name="lyric_input", dtype="float32")
    duration_input = tf.keras.Input(shape=train_duration.shape[1:], name="duration_input")
    notenum_input = tf.keras.Input(shape=train_notenum.shape[1:], name="notenum_input", dtype="float32")

    lyric_embedded = layers.Embedding(input_dim=max_lyric_index + 1, output_dim=128)(lyric_input)

    duration_reshaped = layers.Reshape((784, 1))(duration_input)
    notenum_reshaped = layers.Reshape((784, 1))(notenum_input)


    merged = layers.Concatenate()([lyric_embedded, duration_reshaped, notenum_reshaped])

    lstm_out = layers.LSTM(256, return_sequences=True)(merged)
    lstm_out = layers.LSTM(256, return_sequences=True)(lstm_out)

    pooled_out = layers.AveragePooling1D(pool_size=784 // 128)(lstm_out)[:, :128, :]
    
    output = layers.TimeDistributed(layers.Dense(700, activation='linear'))(pooled_out)

    model = tf.keras.Model(
        [lyric_input, duration_input, notenum_input],
        outputs=output,
        name="vocaloid_model"
    )

    model.compile(
        optimizer='adam', 
        loss='mse',
        metrics=['mae']
    )

    model.fit(
       [train_lyric, train_duration, train_notenum],  # 入力データ（リスト形式）
        y_train,  # 出力データ
        batch_size=32,
        epochs=15,
       validation_split=0.2  # 20%を検証用データに分割
    )

    predictions = model.predict(
        [test_lyric, test_duration, test_notenum],  # 検証用データ
        batch_size=32
    )
    print("Predictions shape:", predictions.shape)

    model.save("../data/model.keras")


2024-12-31 15:38:13.871279: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735627093.885375    2242 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735627093.889976    2242 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-31 15:38:13.904868: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing ust file
parsing ust file...
done parsing u

I0000 00:00:1735627108.360674    2242 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5361 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/15


2024-12-31 15:38:36.486657: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25088000 exceeds 10% of free system memory.
2024-12-31 15:38:36.504022: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25088000 exceeds 10% of free system memory.
I0000 00:00:1735627118.633439    6524 cuda_dnn.cc:529] Loaded cuDNN version 90600


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 918ms/step - loss: 0.2463 - mae: 0.4214 - val_loss: 0.1675 - val_mae: 0.3315
Epoch 2/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 273ms/step - loss: 0.1604 - mae: 0.3262 - val_loss: 0.0859 - val_mae: 0.2376
Epoch 3/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step - loss: 0.0794 - mae: 0.2280 - val_loss: 0.0566 - val_mae: 0.1958
Epoch 4/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 238ms/step - loss: 0.0589 - mae: 0.1994 - val_loss: 0.0548 - val_mae: 0.1907
Epoch 5/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 294ms/step - loss: 0.0556 - mae: 0.1916 - val_loss: 0.0527 - val_mae: 0.1865
Epoch 6/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 262ms/step - loss: 0.0544 - mae: 0.1896 - val_loss: 0.0508 - val_mae: 0.1852
Epoch 7/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 261ms/step - loss: 0.0522 - mae: 0.1872 - val_loss: 0.0501 - val_mae: 0.1835
Epoch 8/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 239ms/step - loss: 0.0490 - mae: 0.1812 - val_loss: 0.0500 - val_mae: 0.1836
Epoch 9/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 233ms/step - loss: 0.0489 - mae: 0.1819 - va

In [111]:
412

512

In [60]:
89600 / 70

1280.0